In [1]:

# (Optional) Install dependencies — uncomment to run in a fresh environment / Colab
# !pip install tensorflow
# !pip install openai


In [2]:

corpus = """Artificial intelligence helps with writing, coding and creativity.
Text generation models learn patterns from examples and can produce coherent paragraphs.
This tiny corpus is only for demo purposes.""".lower()

# Build vocabulary
chars = sorted(list(set(corpus)))
char_to_idx = {c:i for i,c in enumerate(chars)}
idx_to_char = {i:c for c,i in char_to_idx.items()}

print(f"Corpus length: {len(corpus)} characters, vocab size: {len(chars)}")


# Sequence settings
seq_length = 40  # keep small for demo
step = 3

sentences = []
next_chars = []
for i in range(0, len(corpus) - seq_length, step):
    sentences.append(corpus[i: i + seq_length])
    next_chars.append(corpus[i + seq_length])

print(f"Number of training sequences: {len(sentences)}")


# One-hot encoding (small)
import numpy as np
X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, ch in enumerate(sentence):
        X[i, t, char_to_idx[ch]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

# Model definition (Keras)
# NOTE: This cell requires tensorflow installed to run.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(seq_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.summary()

# Train for a tiny number of epochs so it's fast in demos
epochs = 6  # increase for better results
history = model.fit(X, y, batch_size=16, epochs=epochs)


Corpus length: 199 characters, vocab size: 25
Number of training sequences: 53


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 128)            │        78,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 82,073 (320.60 KB)

 Trainable params: 82,073 (320.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 3.1969 
Epoch 2/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3.7512
Epoch 3/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.2574
Epoch 4/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.8743
Epoch 5/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 2.9323
Epoch 6/6
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2.8469


In [3]:

import numpy as np
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(model, seed, length=300, temperature=1.0):
    generated = seed
    sentence = seed[-seq_length:]
    for i in range(length):
        x_pred = np.zeros((1, seq_length, len(chars)))
        for t, ch in enumerate(sentence):
            if ch in char_to_idx:
                x_pred[0, t, char_to_idx[ch]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

seed = corpus[:seq_length]
try:
    print(generate_text(model, seed, length=400, temperature=0.8))
except NameError:
    print("Run the training cell first to build 'model' and train it.")


artificial intelligence helps with writircnnrenpp le pea tes  cedem ep.nrmeefcgddpeeereamicc otcdep eptmeacnacdpycepepeepelr adceoaefcerrpanp eecac colpersreodreor ,aemxeeeecmdsenceeeepedteeredepnesr.tlclmn  eecnnceeeepallevrmerert aeepedeee.lcierep eiepdgepen vetgeierfeeess re pv edcregenm eepr eeaeeeceleadelesmgrca nrehppeeedreerececeeldettpvaevedagacerarereeen anaeccpsrrceme ec teeetedsarleane el ea.eos eeegire.ecprddecpcme  rectec e


In [4]:

# Example GPT usage (skeleton) — requires your OpenAI API key and the 'openai' package.
# Do NOT store API keys in notebooks in plaintext for production — use environment variables or secrets.

# pip install openai
# export OPENAI_API_KEY='your_api_key_here'  # or set in your environment

import os
# import openai
# openai.api_key = os.getenv('OPENAI_API_KEY')

# Example prompt and call (uncomment after installing openai and setting API key)
# response = openai.ChatCompletion.create(
#     model='gpt-4o-mini',  # or another model accessible to you
#     messages=[
#         {'role': 'system', 'content': 'You are a helpful assistant that writes coherent paragraphs.'},
#         {'role': 'user', 'content': 'Write a 3-paragraph overview of renewable energy advantages.'}
#     ],
#     max_tokens=300,
#     temperature=0.7,
# )
# text = response['choices'][0]['message']['content']
# print(text)

print("This is a code skeleton. Uncomment the API call after installing the 'openai' package and setting your OPENAI_API_KEY environment variable.")


This is a code skeleton. Uncomment the API call after installing the 'openai' package and setting your OPENAI_API_KEY environment variable.
